In [15]:
import scrapy
import scrapy.crawler as crawler
from multiprocessing import Process, Queue
from twisted.internet import reactor
import json
import time
import unidecode

import pandas as pd

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

In [16]:
jobs = pd.read_excel('data/all_jobs_150KM.xlsx')
jobs

,job_title,job_title_found,location,company,jobkeys,rome_code,distance
0,adjointe rh,RESPONSABLE ADJOINT DE MAGASIN TOULOUSE H/F,"Toulouse , france",BRIDGE RH ET ASSOCIES,https://www.indeed.fr/voir-emploi?cmp=Bridge-R...,M1501,125
1,adjointe rh,ADJOINT CHEF D'EQUIPE LOGISTIQUE H/F,"Damazan , france",ARMANDIE RH,https://www.indeed.fr/viewjob?jk=9a3b8c2b290700bb,M1501,119
2,adjointe rh,DIRECTEUR COMMERCIAL HYPERMARCHE H/F,"Agen , france",FID RH,https://www.indeed.fr/voir-emploi?cmp=FID-RH&t...,M1501,118
3,adjointe rh,Directeur d'usine H/F,"Mont de Marsan , france",PHI RH,https://www.indeed.fr/voir-emploi?cmp=Phi-Rh&t...,M1501,104
4,adjointe rh,DIRECTEUR ADJOINT,"Toulouse , france",CTI SUD,https://www.indeed.fr/viewjob?jk=44084be3e22d38f4,M1501,125
5,adjointe rh,ADJOINT RAYON FRAIS LIBRE SERVICE HYPERMARCHE H/F,"Saint Sever , france",FID RH,https://www.indeed.fr/voir-emploi?cmp=FID-RH&t...,M1501,91.8
6,adjointe rh,RESPONSABLE ADJOINT DE MODE H/F PAU,"Pau , france",BRIDGE RH ET ASSOCIES,https://www.indeed.fr/voir-emploi?cmp=Bridge-R...,M1501,55.0
7,adjointe rh,Conseiller en insertion professionnelle EREF H/F,"Labastide Saint Pierre , france",Communauté de Communes Grand Sud Tarn-et-Garonne,https://www.indeed.fr/voir-emploi?cmp=Communau...,M1501,129
8,adjointe rh,CHEF DE RAYON BAZAR PERMANENT HYPERMARCHE H/F,"Saint Sever , france",FID RH,https://www.indeed.fr/voir-emploi?cmp=FID-RH&t...,M1501,91.8
9,adjointe rh,Attaché d'administration hospitalière,"Pont du Casse , france",CHD LA CANDELIE,https://www.indeed.fr/voir-emploi?cmp=CHD-LA-C...,M1501,125


In [17]:
companies = jobs['company'].unique()
len(companies)

2258

In [77]:
jobs_relevants = pd.read_csv('data/Danone.xlsx - relevant_companies.csv')
jobs_relevants

,company_name,distance,jobs,location,num_employees,rating,rome_code
0,SATAR,95.8,Opérateur cariste,AGEN,0.0,4.4,N1101
1,AXE TRAVAIL TEMPORAIRE,42.0,Adjointe RH,AUCH,1.0,2.5,M1501
2,O2,78.6,Adjointe RH,MONT,0.0,2.5,M1501
3,MAIRIE,56.7,TECHNICIEN SERVICES GENERAUX,BAREGES,10.0,3.0,I1203
4,MAIRIE,100.0,Adjointe RH,BLAGNAC,0.0,4.2,M1501
5,MANPOWER FRANCE,93.5,Adjointe RH,BOE,3.0,4.4,M1501
6,KDI METAL BARBIER BESSON,92.9,Opérateur cariste,COLOMIERS,20.0,3.4,N1101
7,AIRBUS,98.4,Manager en ressources humaines,BLAGNAC,0.0,2.5,M1503
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,SDIS64,45.1,Adjointe RH,PAU,0.0,4.4,M1501


In [79]:
companies_relevants = jobs_relevants['company_name'].unique()
len(companies_relevants)

1020

# DRIVER

In [18]:
#getdriver local

options = Options()
#options.headless = True
#driver = webdriver.Firefox(options=options, executable_path=r'/Users/neobrain/Projects/machine-learning/python-web-scrapper/neoscrape/driver/mac/geckodriver')
driver = webdriver.Firefox(options=options, executable_path=r'C:/driver/geckodriver.exe')

In [19]:
company_name = companies[1]
company_name

'ARMANDIE RH'

In [20]:
#base_url = 'https://www.societe.com/cgi-bin/recherche'
base_url = 'https://www.societe.com/cgi-bin/liste?nom=COMPANY_NAME&dirig=&pre=&ape=&dep='

In [21]:
driver.get(base_url.replace('COMPANY_NAME', company_name))

In [22]:
search_result = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,
                                                                         "/html/body/div[1]/div[8]/section/div/div/a/div/p[2]/span")))
search_result.click()

In [23]:
tbody = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
    (By.XPATH, "/html/body/div[1]/div[13]/div[1]/div/section/div[2]/table[1]/tbody")))


In [24]:
tbody.text

"Statut RCS Immatriculée au RCS le 20-12-2010\nStatut INSEE Enregistrée à l'INSEE le 06-12-2010\nDénomination ARMANDIE RH\nAdresse 1050 AV DU MIDI 47000 AGEN\nTéléphone\nAfficher le numéro\nSIREN 528 893 076\nSIRET (siege) 52889307600018\nN° de TVA Intracommunautaire\nObtenir le numéro de TVA\n\nVoir PLUS +\nActivité (Code NAF ou APE)\nActivités des agences de travail temporaire (7820Z)\nVoir PLUS +\nForme juridique\nSociété par actions simplifiée\nDate immatriculation RCS 20-12-2010\nStatuts constitutifs\nDate de dernière mise à jour 24-06-2020\nDerniers statuts publiés\nVoir PLUS +\nTranche d'effectif\n6 à 9 salariés\nVoir PLUS +\nCapital social\n110 000,00 €"

In [25]:
try:
    see_more = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,
                                                                         "/html/body/div[1]/div[13]/div[1]/div/section/div[2]/p")))
    see_more.find_element_by_tag_name("a").click()
except:
    pass

In [26]:
naf = ''
date = ''
num_emp = ''
address = ''
for tr in tbody.find_elements_by_tag_name("tr"):
    tds = tr.find_elements_by_tag_name("td")
    for td in tds:
        try:
            #print(unidecode.unidecode(str(tds[0].text).replace(' ', '_').lower()) + ' -> ' + tds[1].text)
            #results.append(td.text)
            if 'Code NAF ou APE' in str(td.text):
                naf = td.text
            if 'Statuts constitutifs' in str(td.text):
                date = td.text
            if "Tranche d'effectif" in str(td.text):
                num_emp = td.text
            if "Adresse" in str(td.text):
                try:
                    address = tds[i+1].text
                except:
                    address = None
        except Exception as e:
             print(e.args)


#results
json = {
    'company': company_name,
    'num_emp': clean_num_emp(num_emp),
    'naf': clean_naf(naf),
    'date': clean_date(date)
}
                
json

NameError: name 'clean_num_emp' is not defined

In [ ]:
naf, num_emp, date

In [31]:
def clean_naf(naf):
    try:
        return naf.split('\n')[2]
    except:
        print("Erro parsing naf: ", naf)
        return None
clean_naf('Voir PLUS +\nActivité (Code NAF ou APE)\nConstruction de véhicules automobiles (2910Z)')
    

'Construction de véhicules automobiles (2910Z)'

In [32]:
def clean_date(date):
    try:
        return date.split('\n')[0]
    except:
        print("Erro parsing date: ", date)
        return None

clean_date('03-02-2011\nStatuts constitutifs')
   

'03-02-2011'

In [33]:
def clean_num_emp(num_emp):
    try:
        return str(num_emp.split('\n')[2])
    except:
        print("Erro parsing num_emp: ", num_emp)
        return None

clean_num_emp("Voir PLUS +\nTranche d'effectif\n10 000 salariés et plus")
   

'10 000 salariés et plus'

In [42]:
def get_infos(company):
    driver.get(base_url.replace('COMPANY_NAME', company))

    search_result = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH,
                                                                             "/html/body/div[1]/div[8]/section/div/div/a/div/p[2]/span")))
    search_result.click()

    tbody = WebDriverWait(driver, 3).until(EC.presence_of_element_located(
        (By.XPATH, "/html/body/div[1]/div[13]/div[1]/div/section/div[2]/table[1]/tbody")))

    try:
        see_more = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH,
                                                                             "/html/body/div[1]/div[13]/div[1]/div/section/div[2]/p")))
        see_more.find_element_by_tag_name("a").click()
    except:
        pass
    
    #
    naf = ''
    date = ''
    num_emp = ''
    address = ''
    for tr in tbody.find_elements_by_tag_name("tr"):
        tds = tr.find_elements_by_tag_name("td")
        for td in tds:
            try:
                #print(unidecode.unidecode(str(tds[0].text).replace(' ', '_').lower()) + ' -> ' + tds[1].text)
                #results.append(td.text)
                if 'Code NAF ou APE' in str(td.text):
                    naf = td.text
                if 'Statuts constitutifs' in str(td.text):
                    date = td.text
                if "Tranche d'effectif" in str(td.text):
                    num_emp = td.text
                if "Adresse" in str(td.text):
                    try:
                        address = tds[i+1].text
                    except:
                        address = None
            except Exception as e:
                 print(e.args)

    
    #results
    return {
        'company': company,
        'num_emp': clean_num_emp(num_emp),
        'naf': clean_naf(naf),
        'date': clean_date(date)
    }
    

In [35]:
get_infos(companies[1])

{'company': 'ARMANDIE RH',
 'num_emp': '6 à 9 salariés',
 'naf': 'Activités des agences de travail temporaire (7820Z)',
 'date': '20-12-2010'}

In [ ]:
companies[1:5]

In [74]:
results = []
erros = []

#10:02
#5min 50
for company in companies[1250:1550]:
    time.sleep(1)
    try:
        print('Collecting info about: ', company)
        
        info = get_infos(company)
        results.append(info)  
        
        if info['naf'] == None:
            erros.append(company)
    except:
        print("ERROR", company)
        erros.append(company)
    
results, erros

ERROR ArchiBat FR
ERROR Auterive Emploi Carrière Chrystelle
ERROR Ariège Emploi Carrière Mélanie
ERROR Solead RH
Erro parsing num_emp:  
ERROR Groupe Cogeser
ERROR 3bconseilrh-accompagnement
ERROR My Retail Agency
ERROR CRCDC OC
ERROR ORKYN'
ERROR ENDEO Recrutement
ERROR CARREDOR
ERROR Axonaut
Erro parsing num_emp:  
ERROR KAEFFER WANNER
ERROR Réseau Primever France
Erro parsing num_emp:  
Erro parsing num_emp:  
ERROR Payjob
Erro parsing num_emp:  
Erro parsing num_emp:  
ERROR Badenoch & Clark
ERROR startpeople
ERROR Le Carré RH
ERROR EHPAD/USLD Le clos Montreuil
Erro parsing num_emp:  
ERROR EURÉCIA
Erro parsing num_emp:  
ERROR LE CARRE RH VILLETTE
Erro parsing num_emp:  
Erro parsing num_emp:  
ERROR OPENSOURCING S.H
Erro parsing num_emp:  
ERROR SANDRA LABARTHE
ERROR Espérance banlieues
Erro parsing num_emp:  
ERROR Cafés Merling
Erro parsing num_emp:  
ERROR Groupe King Jouet
ERROR Würth France
Erro parsing num_emp:  
ERROR Relais & Chateaux
ERROR CERFRANCE Garonne-et-Tarn
ERROR

Erro parsing num_emp:  
ERROR Armée de l'Air
ERROR Tout & Bon Toulouse
Erro parsing num_emp:  
ERROR Lab'Oïkos
ERROR Les voyageurs engagés
Erro parsing num_emp:  
ERROR OFFICINE GRAINE DE PASTEL
ERROR LEC Grand Sud
Erro parsing num_emp:  
Erro parsing num_emp:  
Erro parsing num_emp:  
ERROR Agence clickandbuyauto.fr
ERROR Yosurf-school
ERROR Man's Beard
ERROR Capilog
Erro parsing num_emp:  
ERROR NightsOut
ERROR camping la paillotte
ERROR Gisaïa
ERROR Camping Capfun Le Paradis De Bazas
ERROR WAWA surfschool
ERROR FORMATION LADY BOSS
ERROR ATELIER KATANGA
Erro parsing num_emp:  
ERROR Casino Supermarchés
ERROR Centre hospitalier ' Jean-Pierre CASSABEL '
ERROR Liebherr Group
Erro parsing num_emp:  
ERROR BEN.B SURF SCHOOL
Erro parsing num_emp:  
ERROR CHU PURPAN
ERROR Nature et découvertes
Erro parsing num_emp:  
ERROR SA GERS EQUIPEMENT
Erro parsing num_emp:  
Erro parsing naf:  
ERROR solutions30
ERROR Evoplus
Erro parsing num_emp:  
Erro parsing num_emp:  
Erro parsing num_emp:  
ERR

([{'company': 'Emergences RH',
   'num_emp': '1 à 2 salariés',
   'naf': 'Conseil pour les affaires et autres conseils de gestion (7022Z)',
   'date': '16-10-2001'},
  {'company': 'Persuaders RH',
   'num_emp': '10 à 19 salariés',
   'naf': 'Conseil pour les affaires et autres conseils de gestion (7022Z)',
   'date': '27-10-2004'},
  {'company': 'NEXTGEN RH',
   'num_emp': '20 à 49 salariés',
   'naf': 'Conseil pour les affaires et autres conseils de gestion (7022Z)',
   'date': '08-06-2009'},
  {'company': 'Pharmagest',
   'num_emp': '500 à 999 salariés',
   'naf': 'Conseil en systèmes et logiciels informatiques (6202A)',
   'date': '25-01-1996'},
  {'company': 'Autogrill',
   'num_emp': '50 à 99 salariés',
   'naf': 'Restauration de type rapide (5610C)',
   'date': '28-10-2016'},
  {'company': 'Toujas et Coll',
   'num_emp': '100 à 199 salariés',
   'naf': 'Commerce de gros (commerce interentreprises) de bois et de matériaux de construction (4673A)',
   'date': '28-10-1954'},
  {'com

In [75]:
results = pd.DataFrame(results)
results

,company,date,naf,num_emp
0,Emergences RH,16-10-2001,Conseil pour les affaires et autres conseils d...,1 à 2 salariés
1,Persuaders RH,27-10-2004,Conseil pour les affaires et autres conseils d...,10 à 19 salariés
2,NEXTGEN RH,08-06-2009,Conseil pour les affaires et autres conseils d...,20 à 49 salariés
3,Pharmagest,25-01-1996,Conseil en systèmes et logiciels informatiques...,500 à 999 salariés
4,Autogrill,28-10-2016,Restauration de type rapide (5610C),50 à 99 salariés
5,Toujas et Coll,28-10-1954,Commerce de gros (commerce interentreprises) d...,100 à 199 salariés
6,Cap RH,,Autres organisations fonctionnant par adhésion...,None
7,Toulouse.fr,27-09-2002,Commerce de détail d'habillement en magasin sp...,Unités non employeuses (pas de salarié au cour...
8,AGES SANS FRONTIERES,,Hébergement social pour personnes âgées (8730A),200 à 249 salariés
9,L'ANNEXE RH,25-03-2009,Conseil pour les affaires et autres conseils d...,0 salarié (unités ayant eu des salariés au cou...


In [76]:
results.to_clipboard(index=False, header=False)

In [80]:
companies_relevants

array(['SATAR', 'AXE TRAVAIL TEMPORAIRE', 'O2', ..., 'SGS FRANCE',
       'TESTIA', 'ECA GROUP'], dtype=object)

In [81]:
results = []
erros = []

#10:02
#5min 50
for company in companies_relevants[1:500]:
    time.sleep(1)
    try:
        print('Collecting info about: ', company)
        
        info = get_infos(company)
        results.append(info)  
        
        if info['naf'] == None:
            erros.append(company)
    except:
        print("ERROR", company)
        erros.append(company)
    
results, erros

Erro parsing num_emp:  
Erro parsing num_emp:  
ERROR nan
ERROR SDIS64
Erro parsing num_emp:  
ERROR BATI + 65
Erro parsing num_emp:  
ERROR COMMUNAUTE DE COMMUNES DES HAUTS TOLOSAN
ERROR PYRENEES PARTA 'GE
ERROR LA GRANDE BRASSERIE DE LA GARE
ERROR MAIRIE TARBES/CAISSE DES ECOLES
ERROR ALBRET COMMUNAUTÉ
ERROR MAIRIE DE HAGETMAU
ERROR PETR PAYS LOURDES VALLEES GAVES
ERROR HOTEL SAINT SAUVEUR***
ERROR CENTRE GESTION FONCTION PUB TERRITORIALE
ERROR INSTITUT SAINT SIMON
Erro parsing num_emp:  
ERROR SOC D EXPLOITATION THERMIQUE DU MIRAIL
ERROR COMPASS GROUP FRANCE ENSEIGNEMENT SANTE
Erro parsing num_emp:  
Erro parsing num_emp:  
Erro parsing num_emp:  
ERROR SII SOC POUR L INFORMATIQUE INDUSTRIEL
ERROR B & T ASSOCIES
ERROR TPM AEROMONTAGE
ERROR FEDERATION DEPARTEMENTALE PYRENE PLUS
ERROR JOL ENERGY
ERROR SCOLAREST 2529
Erro parsing num_emp:  
Erro parsing num_emp:  
ERROR ZODIAC CABIN INTERIORS EUROPE
ERROR INST NAT FORMATION RECHERCHES EDUC PER
Erro parsing num_emp:  
ERROR SARL AERO PR

Erro parsing num_emp:  
Erro parsing num_emp:  
ERROR DELEGATION DEPART HAUTE GARONNE
ERROR COMMUNAUTE COM DE HAUTE BIGORRE
Erro parsing num_emp:  
ERROR HERTZ FRANCE AT
Erro parsing num_emp:  
Erro parsing num_emp:  
ERROR DERICHEBOURG SOURCING AERO & ENERGY
Erro parsing num_emp:  
ERROR CTE COMMUNES GASCOGNE TOULOUSAINE
Erro parsing num_emp:  
Erro parsing num_emp:  
Erro parsing num_emp:  
ERROR RESEAU EXPER GERSOIS D AIDE REINSERTION
ERROR CR INSERTION
Erro parsing num_emp:  
ERROR TECHNI MOULAGES DIVISION DE TEC
ERROR ONET TECHNOLOGIES TI GOLFECH
Erro parsing num_emp:  
ERROR SODEXHO FRANCE ENTREPRISES ADMINISTRAT
ERROR GE DES INDUSTRIES D'OC TOULOUSE
Erro parsing num_emp:  
ERROR SONOVISION ITEP
ERROR CMT+
ERROR ORTET LOCATION TANSPORT SERVICE
Erro parsing num_emp:  
ERROR CAMPING DE L'ARROS
ERROR AEROCONSEIL SA
ERROR COMMUNAUTÉ COMMUNES COUSERANS PYRÉNÉES
ERROR MAC DONALD'S AGEN SUD
Erro parsing num_emp:  
ERROR MAIRIE LABASTIDETTE
ERROR LABO RECHERCHE ADD ALIMENTAIRE
Erro parsi

ERROR CABINET D'EXPERTISES FERDINAND RESE
Erro parsing num_emp:  
ERROR CÔTE ET MER
Erro parsing num_emp:  
Erro parsing num_emp:  
ERROR HOURAT SARL
Erro parsing num_emp:  
Erro parsing num_emp:  
ERROR CAMPING D 'ARROUACH
ERROR T.R. EXPRESS TRANSPORTS RAPIDES
ERROR SODEXHO FRANCE ENTREPRISES ADMIN
ERROR MIROITERIE MENUISERIE 12
ERROR BIOLANDES SA
ERROR GROUPE CRESCENDO RESTAURATION
ERROR GUY DALBOS AUTOMATISMES & LOGISTIQUE
ERROR ACIA AERO TECHNICS CONVERSIONS & LEASING
ERROR EFS OCCITANIE
Erro parsing num_emp:  
Erro parsing num_emp:  
ERROR ETABLISSEMENT DANIEL COUSSO
ERROR LA TABLE DES CORNIERES
Erro parsing num_emp:  
Erro parsing num_emp:  
ERROR SOCIETE BARRERE YANNICK ET R GIS
Erro parsing num_emp:  
ERROR FOYER DE VIE LES THUYAS
ERROR SAINT ELOI MECANIQUE OUTILLAGE
ERROR LINDT ET SPRUNGLI
ERROR RELAIS DU BOIS PERCHE
ERROR REF INTERNE 002636
ERROR SURETE AEROPORTUAIRE ICTS FRANCE
ERROR SARL EPIN SECURITE PRIVEE
ERROR SYNDICAT COPROPRIETAIRES LES HESPERIDES
Erro parsing num_emp

Erro parsing num_emp:  
Erro parsing num_emp:  
Erro parsing num_emp:  
Erro parsing num_emp:  
ERROR INTERVENTION TELESURV.SECURITE SYSTEME
ERROR CCI DE HAUTE GARONNE SITE DE BLAGNAC
Erro parsing num_emp:  


([{'company': 'AXE TRAVAIL TEMPORAIRE',
   'num_emp': "Unités non employeuses (pas de salarié au cours de l'année de référence et pas d'effectif au 31/12).",
   'naf': 'Activités des agences de travail temporaire (7820Z)',
   'date': '15-03-1999'},
  {'company': 'O2',
   'num_emp': None,
   'naf': 'Autres organisations fonctionnant par adhésion volontaire (9499Z)',
   'date': ''},
  {'company': 'MAIRIE',
   'num_emp': None,
   'naf': 'Restauration de type rapide (553B)',
   'date': '10-03-1983'},
  {'company': 'MANPOWER FRANCE',
   'num_emp': '2000 à 4999 salariés',
   'naf': 'Activités des agences de travail temporaire (7820Z)',
   'date': '12-06-1997'},
  {'company': 'KDI METAL BARBIER BESSON',
   'num_emp': '1000 à 1999 salariés',
   'naf': 'Commerce de gros (commerce interentreprises) de minerais et métaux (4672Z)',
   'date': '06-12-1989'},
  {'company': 'AIRBUS',
   'num_emp': "Unités non employeuses (pas de salarié au cours de l'année de référence et pas d'effectif au 31/12).",


In [83]:
pd.DataFrame(results).to_clipboard(index=False)